In [ ]:
%load_ext autoreload
%autoreload 2

import sys


In [ ]:
import os
import array

import pandas
import numpy

import plotly.express
import librosa
from matplotlib import pyplot as plt

from software.dataset.hussain2021 import load_data, load_meta
from firmware.process import GravitySplitter

In [ ]:

dataset_path = './data/hx5kkkbr3j-1'
samplerate = 50

In [ ]:

data = load_data(dataset_path)
data.head()

In [ ]:
meta = load_meta(data)
meta

In [ ]:

def extract_data(data, meta):
    # only accelerometer data
    acc = data.dropna(subset=['acc_x', 'acc_y', 'acc_z']).drop(columns=['mag_x', 'mag_y', 'mag_z', 'gyro_x', 'gyro_y', 'gyro_z'])
    acc = acc.reset_index()
    acc = pandas.merge(acc, meta, left_on='filename', right_on='filename')
    # Setting 2 has more specific protocol
    # Pause for a few seconds in between different regions and bring the brush to a reference point
    acc = acc[acc.setting == 'S2'] 
    # Choose location mounted on brush
    acc = acc[acc.sensor_location == 'A']
    # Choose only manual brushing, not electric
    acc = acc[acc.brush == 'M']

    return acc

acc = extract_data(data, meta)

In [ ]:
def resample(df, freq='1min', func='mean', group='filename', time='time', numeric_only=True):
    grouped = df.reset_index().set_index(time).groupby(group, observed=True).resample(freq)
    out = grouped.agg(func, numeric_only=numeric_only).reset_index().set_index([group, time])
    return out

# Resample to our samplerate
acc_re = resample(acc, freq=pandas.Timedelta(1/samplerate, unit='s')).reset_index()
acc_re

In [ ]:
acc_re.filename.nunique()

In [ ]:

# TODO: use micropython to process
def process(df, sr, bandpass_filter=None, groupby='filename', columns=['acc_x', 'acc_y', 'acc_z'], time='time'):

    def process_timeseries(g, splitter=None):
        g = g.sort_values(time)
        
        splitter = GravitySplitter(samplerate=sr)

        out = []
        index_values = []
        for idx, row in g.iterrows():
            xyz = array.array('f', row[columns])
            splitter.process(xyz)
        
            values = list(splitter.gravity) + list(splitter.motion)
            out.append(values)
            index_values.append(idx)

        names = ['gravity_x', 'gravity_y', 'gravity_z', 'motion_x', 'motion_y', 'motion_z']
        o = pandas.DataFrame(out, columns=names)
        o.index = g.index

        # apply bandpass
        bandpass_columns = ['motion_x', 'motion_y', 'motion_z']
        if bandpass_filter is not None:
            from scipy.signal import sosfilt
            for c in bandpass_columns:
                o[f'bandpass_{c}'] = sosfilt(bandpass_filter, o[c])
        
        print('pp', len(o))
        return o
    
    groups = df.groupby(groupby, as_index=False)
    print('groups', len(groups))
    f = groups.apply(process_timeseries, include_groups=False)
    f.index = f.index.droplevel(0)
    #print(df.head())
    #print('sss\n\n')
    #print(f.head())
    
    out = pandas.merge(df, f, left_index=True, right_index=True)
    return out

ss = pandas.Series(acc_re.filename.unique()).sample(n=42, random_state=1)
sub = acc_re[acc_re.filename.isin(ss)]
pp = process(sub.set_index(['filename', 'time']), sr=samplerate, bandpass_filter=sos)
pp

In [ ]:
def vector_magnitude(vectors, axis=None):
    """Compute the magnitude of multi-dimensional vectors"""
    # alternative is numpy.linalg.norm(vectors)
    mag = numpy.linalg.norm(vectors, axis=axis)
    return mag


In [ ]:
mm = pp.reset_index()
mm = pandas.merge(mm, meta, left_on='filename', right_on='filename')
mm['motion_mag'] = vector_magnitude(mm[['motion_x', 'motion_y', 'motion_z']], axis=1)
mm['bandpass_motion_mag'] = vector_magnitude(mm[['bandpass_motion_x', 'bandpass_motion_y', 'bandpass_motion_z']], axis=1)
mm

In [ ]:

for filename, df in mm.groupby('filename'):

    # TODO: also plot motion
    plotly.express.line(df, 
                        x='elapsed',
                        y=['gravity_x', 'gravity_y', 'gravity_z'],
                        facet_row='filename',
                        height=200,
                        width=1000,
                       )


In [ ]:
def make_spectrogram(df, sr=50, hop_length=16, columns=None):
    if columns is None:
        columns = ['acc_x', 'acc_y', 'acc_z']
    df = df.sort_values('elapsed')
    df = df.dropna(subset=columns)
    
    n_fft = 4*hop_length

    # Make combined XYZ spectrogram, concat on frequency axis
    ss = []
    for c in columns:
        v = df[c].values
        S = librosa.stft(v, n_fft=n_fft, hop_length=hop_length)
        S = numpy.abs(S)
        S = librosa.amplitude_to_db(S)
        #S = S[1:,:] # drop DC
        ss.append(S)

    o = numpy.concatenate(ss)
    return o

specs = mm.groupby('filename').apply(make_spectrogram, include_groups=False).to_frame().rename(columns={0:'spectrogram'})
specs;

In [ ]:

# TODO: plot bandpassed motion
# TODO: plot orientation distance from stationary point
for filename, spec in specs.iterrows():
    #print(filename)
    fig, ax = plt.subplots(1, figsize=(20, 5))
    S = spec.spectrogram
    N = numpy.quantile(S, q=0.10, axis=1, keepdims=True)
    #S = S - N
    #print(S.shape, N.shape)
    librosa.display.specshow(ax=ax, data=S)


In [ ]:
def to_spectrum(S):
    s = numpy.mean(S, axis=1)
    o = pandas.Series(s, index=numpy.arange(len(s)))
    return o

def compute_spectrums(df, sr, metric='motion', axes=['x', 'y', 'z']):
    dfs = []
    
    for axis in axes:
        c = f'{metric}_{axis}'

        freqs = librosa.fft_frequencies(n_fft=4*16, sr=50)
    
        specs = df.groupby('filename').apply(make_spectrogram, include_groups=False, columns=[c]).to_frame().rename(columns={0:'spectrogram'})
        ss = specs.spectrogram.apply(to_spectrum)
        ss.columns = freqs

        # Normalize
        N = numpy.mean(ss.values, axis=1, keepdims=1)
        ss = pandas.DataFrame(ss-N, columns=ss.columns, index=ss.index)
        
        sf = pandas.melt(ss.reset_index(), id_vars='filename', var_name='frequency')
        sf['axis'] = axis
        
        dfs.append(sf)

    out = pandas.concat(dfs)
    return out

spectrums = compute_spectrums(mm, sr=samplerate, metric='motion')
#spectrums

fig = plotly.express.line(spectrums,
                    facet_row='axis',
                    color='filename',
                    x='frequency',
                    y='value',
                    height=800,
                    width=1500,
)

lower = 2.0
upper = 5.0
fig.add_vrect(x0=lower, x1=upper, line_width=0, fillcolor="green", opacity=0.2, layer='below')


In [ ]:

mag_spectrums = compute_spectrums(mm, metric='motion', axes=['mag'], sr=samplerate)
#spectrums

fig = plotly.express.line(mag_spectrums,
                    facet_row='axis',
                    color='filename',
                    x='frequency',
                    y='value',
                    height=800,
                    width=1500,
)
fig.add_vrect(x0=lower, x1=upper, line_width=0, fillcolor="green", opacity=0.2, layer='below')



In [ ]:

def design_bandpass(lower=2.0, upper=5.0, sr=100, order=2):
    from scipy.signal import iirfilter
    
    sos = iirfilter(order, [lower, upper], btype='band',
                    analog=False, fs=sr, output='sos',
                    ftype='cheby1', rp=2.0)

    return sos

def plot_bandpass(sos, lower, upper, fmin=0.1, sr=50):

    from scipy.signal import sosfreqz
    import numpy as np
    
    w, h = sosfreqz(sos, 10000, fs=sr)
    fig, ax = plt.subplots(1)
    ax.semilogx(w, 20 * np.log10(np.maximum(abs(h), 1e-5)))
    
    ax.set_title('Chebyshev Type II bandpass frequency response')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Amplitude [dB]')
    ax.axis((fmin, sr/2.0, -100, 10))
    ax.grid(which='both', axis='both')
    for f in [lower, upper]:
        ax.axvline(f, ls='--', alpha=0.5, color='black')
    return fig


sos = design_bandpass(lower=lower, upper=upper, sr=samplerate, order=2)
plot_bandpass(sos, lower=lower, upper=upper, sr=samplerate);

In [ ]:
fig = plotly.express.scatter(mm, 
                    x='gravity_y',
                    y='gravity_z',
                    color='filename',
                    height=800,
                    width=800,
                    opacity=0.2,
                   )
fig.update_traces(marker=dict(size=5.0))
fig.update_layout(showlegend=False)

In [ ]:
fig = plotly.express.scatter_matrix(mm.sample(frac=0.05),
    dimensions=["motion_x", "motion_y", "motion_z"],
    color="subject",
    height=1200,
    width=1200,
    opacity=0.2,
)
fig.update_traces(marker=dict(size=3.0))
fig.show()